In [21]:
from pydantic_ai import Agent

In [22]:
import os

# Add parent directory to Python path so we can import src
import sys

sys.path.append(os.path.abspath(".."))

In [23]:
documents_produced_agent = Agent[str, str](
    model="openai:gpt-4o",
    result_type=str,
    retries=3,
)

In [33]:
documents_produced_agent.capture_run_messages

AttributeError: 'Agent' object has no attribute 'capture_run_messages'

In [ ]:
from src.utils import log_exception


# def log_exception(
#     job_id: str = None,
#     model: str = None,
#     error_message: str = None,
#     error_category: str = None,
#     error_traceback: str = None,
# ) -> None:

In [4]:
from typing import Optional, Any, Union, get_origin, get_args
from pydantic import BaseModel, create_model


def create_error_model(model: Any) -> Any:
    """Convert all fields of a Pydantic model to Optional and adds error fields."""
    if not issubclass(model, BaseModel):
        raise ValueError("model must be a subclass of BaseModel")

    fields = {}
    for field_name, field in model.model_fields.items():
        annotation = field.annotation
        # Check if field is already optional
        if get_origin(annotation) is not Union or type(None) not in get_args(
            annotation
        ):
            fields[field_name] = (Optional[field.annotation], None)
        else:
            fields[field_name] = (field.annotation, None)

    NewModel = create_model(
        model.__name__ + "Optional",
        __base__=model,
        error_message=(Optional[str], None),
        error_traceback=(Optional[str], None),
        **fields,
    )
    return NewModel

In [17]:
import functools
import time
import traceback
from typing import Any, Callable, get_type_hints, cast

# A generic type for functions
F = Callable[..., Any]


def handle_llm_errors(
    task_name: str, error_category: str, default_message: Optional[str] = None
):
    """
    Decorator for handling errors in LLM operations.

    This decorator catches exceptions, logs them, and returns a default value
    constructed from the function's annotated return type (which must be a subclass of BaseModel
    or a string).

    Args:
        task_name: Name of the task being performed (for logging).
        error_category: Category of the error (for classification).
        default_message: Default error message if none is provided.

    Returns:
        A decorator function.
    """

    def decorator(func: F) -> F:
        # Get type hints from the function
        hints = get_type_hints(func)
        return_type = hints.get("return", None)

        # Ensure that the return type is a subclass of pydantic.BaseModel or str
        is_str_return = isinstance(return_type, type) and issubclass(return_type, str)

        if not return_type or not (
            is_str_return
            or (isinstance(return_type, type) and issubclass(return_type, BaseModel))
        ):
            raise TypeError(
                "The wrapped function must have a return type annotation that is either a subclass of pydantic.BaseModel or str"
            )

        @functools.wraps(func)
        def wrapper(self, *args: Any, **kwargs: Any) -> Any:
            start_time = time.perf_counter()
            try:
                result = func(self, *args, **kwargs)
                # Log task completion if the logger is available
                if hasattr(self, "log_task_completion"):
                    self.log_task_completion(task_name, start_time)
                return result
            except Exception as e:
                # Construct error message
                error_message = (
                    f"{e}: {default_message}, "
                    f"\nCalling {func.__name__} with args: {args}, kwargs: {kwargs} "
                    f"\nStack: {traceback.format_exc()}"
                )
                log_exception(
                    job_id=getattr(self, "job_id", None),
                    model="OpenAIModel_GPT_4O",
                    error_message=error_message,
                    error_category=error_category,
                    error_traceback=str(e),
                )

                # Handle different return types
                if is_str_return:
                    # For string returns, just return a friendly error message
                    return f"An error occurred: {error_message}. Details: {str(e)}"
                else:
                    # For Pydantic models, return a default instance with added error info
                    error_model = create_error_model(return_type)
                    error_info = {
                        "error_message": error_message,
                        "error_traceback": str(e),
                    }
                    return error_model.model_validate(error_info)

        return cast(F, wrapper)

    return decorator

In [18]:
# Example usage:


class MyResponseModel(BaseModel):
    result: Optional[str] = None


class MyLLMClient:
    job_id = "12345"

    def log_task_completion(self, task_name: str, start_time: float):
        elapsed = time.perf_counter() - start_time
        print(f"Task {task_name} completed in {elapsed:.2f} seconds")

    def some_log_exception(self, **kwargs):
        print("Logging exception:", kwargs)

    # Simulate your log_exception function here
    log_exception = staticmethod(lambda **kwargs: print("Logged:", kwargs))

    @handle_llm_errors("process data", "processing_error")
    def process_data(self, data: str) -> MyResponseModel:
        # Simulate an error
        if data == "fail":
            raise ValueError("Simulated failure")
        return MyResponseModel(result=data.upper())


# Testing
client = MyLLMClient()
print(client.process_data("hello"))
print(client.process_data("fail"))

2025-03-02 16:33:57,350 - src.utils - ERROR - Error in job 12345, model OpenAIModel_GPT_4O, category processing_error: Simulated failure: None, 
Calling process_data with args: ('fail',), kwargs: {} 
Stack: Traceback (most recent call last):
  File "C:\Users\pdoub\AppData\Local\Temp\ipykernel_3736\1614251534.py", line 45, in wrapper
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\pdoub\AppData\Local\Temp\ipykernel_3736\2678870425.py", line 24, in process_data
    raise ValueError("Simulated failure")
ValueError: Simulated failure



Task process data completed in 0.00 seconds
result='HELLO'
result=None error_message='Simulated failure: None, \nCalling process_data with args: (\'fail\',), kwargs: {} \nStack: Traceback (most recent call last):\n  File "C:\\Users\\pdoub\\AppData\\Local\\Temp\\ipykernel_3736\\1614251534.py", line 45, in wrapper\n    result = func(self, *args, **kwargs)\n             ^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "C:\\Users\\pdoub\\AppData\\Local\\Temp\\ipykernel_3736\\2678870425.py", line 24, in process_data\n    raise ValueError("Simulated failure")\nValueError: Simulated failure\n' error_traceback='Simulated failure'


In [19]:
t = client.process_data("fail")

2025-03-02 16:34:35,825 - src.utils - ERROR - Error in job 12345, model OpenAIModel_GPT_4O, category processing_error: Simulated failure: None, 
Calling process_data with args: ('fail',), kwargs: {} 
Stack: Traceback (most recent call last):
  File "C:\Users\pdoub\AppData\Local\Temp\ipykernel_3736\1614251534.py", line 45, in wrapper
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\pdoub\AppData\Local\Temp\ipykernel_3736\2678870425.py", line 24, in process_data
    raise ValueError("Simulated failure")
ValueError: Simulated failure



In [20]:
t.model_dump()

{'result': None,
 'error_message': 'Simulated failure: None, \nCalling process_data with args: (\'fail\',), kwargs: {} \nStack: Traceback (most recent call last):\n  File "C:\\Users\\pdoub\\AppData\\Local\\Temp\\ipykernel_3736\\1614251534.py", line 45, in wrapper\n    result = func(self, *args, **kwargs)\n             ^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "C:\\Users\\pdoub\\AppData\\Local\\Temp\\ipykernel_3736\\2678870425.py", line 24, in process_data\n    raise ValueError("Simulated failure")\nValueError: Simulated failure\n',
 'error_traceback': 'Simulated failure'}